# Proyecto UT5:
## Predicción del tipo de un pokémon a partir de su imagen

### Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras as kr
from tensorflow.keras.layers import Normalization, IntegerLookup, StringLookup

import os
import shutil

### Data

Primero preparamos los pokemon con sus tipos

In [2]:
dataframe = pd.read_csv("pokemon.csv")
dataframe

,Name,Type1,Type2
0,bulbasaur,Grass,Poison
1,ivysaur,Grass,Poison
2,venusaur,Grass,Poison
3,charmander,Fire,NaN
4,charmeleon,Fire,NaN
...,...,...,...
804,stakataka,Rock,Steel
805,blacephalon,Fire,Ghost
806,zeraora,Electric,NaN
807,meltan,Steel,NaN


Asignamos las clases al tipo 1

In [3]:
clases = dataframe.Type1.unique()
clases

array(['Grass', 'Fire', 'Water', 'Bug', 'Normal', 'Poison', 'Electric',
       'Ground', 'Fairy', 'Fighting', 'Psychic', 'Rock', 'Ghost', 'Ice',
       'Dragon', 'Dark', 'Steel', 'Flying'], dtype=object)

Guardamos el path de la imagen del pokemon en el dataframe.

In [4]:
# import the modules
images = []
# get the path or directory
folder_dir = "images"
for image in os.listdir(folder_dir):
    # check if the image end swith png or jpg or jpeg
    if (image.endswith(".png") or image.endswith(".jpg") or image.endswith(".jpeg")):
        # display
        images.append(image)


In [5]:
paths = []
format = []
for d in dataframe.Name:
    for i in images:
        if (d + ".png" == i or d + ".jpg" == i):
            paths.append("images/"+i)
            if (d + ".png" == i):
                format.append(".png")
            else:
                format.append(".jpg")

In [6]:
dataframe = dataframe.assign(ImgPath=paths)
dataframe = dataframe.assign(Format=format)

dataframe

,Name,Type1,Type2,ImgPath,Format
0,bulbasaur,Grass,Poison,images/bulbasaur.png,.png
1,ivysaur,Grass,Poison,images/ivysaur.png,.png
2,venusaur,Grass,Poison,images/venusaur.png,.png
3,charmander,Fire,NaN,images/charmander.png,.png
4,charmeleon,Fire,NaN,images/charmeleon.png,.png
...,...,...,...,...,...
804,stakataka,Rock,Steel,images/stakataka.jpg,.jpg
805,blacephalon,Fire,Ghost,images/blacephalon.jpg,.jpg
806,zeraora,Electric,NaN,images/zeraora.jpg,.jpg
807,meltan,Steel,NaN,images/meltan.jpg,.jpg


Creamos el directorio organizado y copiamos las imagenes según su clase

In [7]:
os.mkdir("imageset")
for clase in clases:
    os.mkdir("imageset/" + clase)

for (path, type, name, format) in zip(dataframe.ImgPath, dataframe.Type1, dataframe.Name, dataframe.Format):
    shutil.copyfile(path, "imageset/" + type + "/" + name + format)

Finalmente cargamos las imagenes ya organizadas por tipos

In [8]:
pokemons = tf.keras.utils.image_dataset_from_directory(directory="imageset")

Found 809 files belonging to 18 classes.


In [9]:
pokemons.class_names

['Bug',
 'Dark',
 'Dragon',
 'Electric',
 'Fairy',
 'Fighting',
 'Fire',
 'Flying',
 'Ghost',
 'Grass',
 'Ground',
 'Ice',
 'Normal',
 'Poison',
 'Psychic',
 'Rock',
 'Steel',
 'Water']

In [10]:
# pokemons_val = pokemons.sample_from_datasets(datasets=pokemons, weights=0.2)
# pokemons_train = pokemons.drop(pokemons_val.index)

### Modelo

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation="relu"),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(18)
])

In [16]:
model.compile(
    optimizer='adam',
    loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

In [17]:
model.fit(
    pokemons,
    epochs=10
)

26/26 [==============================] - 13s 473ms/step - loss: 2083.3169 - accuracy: 0.1038


In [19]:
predictions = model.predict(dataframe.ImgPath)

ValueError: in user code:

    File "C:\Users\BDIA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\engine\training.py", line 1801, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\BDIA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\engine\training.py", line 1790, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\BDIA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\engine\training.py", line 1783, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\BDIA\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\keras\engine\training.py", line 1751, in predict_step
 